<img align="right" src="../../additional_data/banner_siegel.png" style="width:1000px;">

# Xarray-I: Data Structure 

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Prerequisites**: There is no prerequisite learning required.


## Background

In the previous notebook, we experienced that the data we wanna access are loaded in a form called **`xarray.dataset`**. This is the form in which earth observation data are usually stored in a datacube.

**`xarray`** is an open source project and Python package which offers a toolkit for working with ***multi-dimensional arrays*** of data. **`xarray.dataset`** is an in-memory representation of a netCDF (network Common Data Form) file. Understanding the structure of a **`xarray.dataset`** is the key to enable us work with these data. Thus, in this notebook, we are mainly dedicated to helping users of our datacube understand its data structure.

Firstly let's come to the end stage of the previous notebook, where we have loaded a data product. The data product "s2_l2a_bavaria" is used as example in this notebook.

## Description

The following topics are convered in this notebook:
* **What is inside a `xrray.dataset` (the structure)?**
* **(Basic) Subset Dataset / DataArray**
* **Reshape a Dataset**

In [1]:
import datacube
import pandas as pd
from odc.ui import DcViewer 
from odc.ui import with_ui_cbk
import xarray as xr
import matplotlib.pyplot as plt

# Set config for displaying tables nicely
# !! USEFUL !! otherwise parts of longer infos won't be displayed in tables
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

# Connect to DataCube
# argument "app" --- user defined name for a session (e.g. choose one matching the purpose of this notebook)
dc = datacube.Datacube(app = "nb_understand_ndArrays")

Failed to resolve driver datacube.plugins.index::default
Error was: ContextualVersionConflict(click 8.0.3 (/tmp/4/.local/lib/python3.8/site-packages), Requirement.parse('click<8,>=4.0'), {'cligj', 'rasterio'})


In [2]:
# Load Data Product
ds = dc.load(product= "s2_l2a_saldi_02",
               x= (20.53, 20.55),
               y= (-28.75, -28.74),
             time = ("2020-10-01", "2020-12-31"),
             group_by = "solar_day",
            progress_cbk=with_ui_cbk())

ds

Failed to resolve driver datacube.plugins.io.read::netcdf
Error was: ContextualVersionConflict(click 8.0.3 (/tmp/4/.local/lib/python3.8/site-packages), Requirement.parse('click<8,>=4.0'), {'cligj', 'rasterio'})


<xarray.Dataset>
Dimensions:          (time: 19, x: 197, y: 112)
Coordinates:
  * time             (time) datetime64[ns] 2020-10-02T08:48:25.419626 ... 202...
  * y                (y) float64 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x                (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref      int32 32734
Data variables: (12/13)
    coastal_aerosol  (time, y, x) int16 222 222 222 222 192 ... 287 287 287 289
    blue             (time, y, x) int16 16 8 8 1 1 12 ... 551 510 386 453 455
    green            (time, y, x) int16 203 219 213 254 243 ... 1015 858 868 869
    red              (time, y, x) int16 148 160 120 118 124 ... 1042 693 861 913
    red_edge1        (time, y, x) int16 450 450 509 509 ... 1753 1500 1500 1278
    red_edge2        (time, y, x) int16 738 738 442 442 ... 2971 3150 3150 3121
    ...               ...
    nir              (time, y, x) int16 170 159 1 1 40 ... 3708 3934 3780 3492
    narrow_nir       (time, y, x) int16 998 998 874 874 ... 3694 3927 3927 3958
    water_vapour     (time, y, x) int16 1862 1862 1862 1862 ... 4119 4119 4294
    swir1            (time, y, x) int16 711 711 742 742 ... 2919 2796 2796 2342
    swir2            (time, y, x) int16 471 471 431 431 ... 1888 1714 1714 1368
    scl              (time, y, x) uint8 2 2 2 2 2 2 2 2 6 ... 4 4 4 4 4 4 4 4 4
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

In [ ]:
#da = ds.to_array().rename({"variable":"band"})
#print(da)

In [ ]:
#ds2 = da.to_dataset(dim="time")
#ds2

## **What is inside a `xarray.dataset`?**
The figure below is a diagramm depicting the structure of the **`xarray.dataset`** we've just loaded. Combined with the diagramm, we hope you may better interpret the texts below explaining the data strucutre of a **`xarray.dataset`**.

![xarray data structure](https://live.staticflickr.com/65535/51083605166_70dd29baa8_k.jpg)

As read from the output block, this dataset has three ***Data Variables*** , "blue", "green" and "red" (shown with colors in the diagramm), referring to individual spectral band.

Each data variable can be regarded as a **multi-dimensional *Data Array*** of same structure; in this case, it is a **three-dimensional array** (shown as 3D Cube in the diagramm) where `time`, `x` and `y` are its ***Dimensions*** (shown as axis along each cube in the diagramm).

In this dataset, there are 35 ***Coordinates*** under `time` dimension, which means there are 35 time steps along the `time` axis. There are 164 coordinates under `x` dimension and 82 coordinates under `y` dimension, indicating that there are 164 pixels along `x` axis and 82 pixels along `y` axis.

As for the term ***Dataset***, it is like a *Container* holding all the multi-dimensional arrays of same structure (shown as the red-lined box holding all 3D Cubes in the diagramm).

So this instance dataset has a spatial extent of 164 by 82 pixels at given lon/lat locations, spans over 35 time stamps and 3 spectral band.

**In summary, *`xarray.dataset`* is substantially a container for high-dimensional *`DataArray`* with common attributes (e.g. crs) attached**, :
* **Data Variables (`values`)**: **it's generally the first/highest dimension to subset from a high dimensional array.** Each `data variable` contains a multi-dimensional array of all other dimensions.
* **Dimensions (`dims`)**: other dimensions arranged in hierachical order *(e.g. 'time', 'y', 'x')*.
* **Coordinates (`coords`)**: Coordinates along each `Dimension` *(e.g. timesteps along 'time' dimension, latitudes along 'y' dimension, longitudes along 'x' dimension)*
* **Attributes (`attrs`)**: A dictionary(`dict`) containing Metadata.

Now let's deconstruct the dataset we have just loaded a bit further to have things more clarified!:D

* **To check existing dimensions of a dataset**

In [3]:
ds.dims

Frozen(SortedKeysDict({'time': 19, 'y': 112, 'x': 197}))

* **To check the coordinates of a dataset**

In [4]:
ds.coords#['time']

Coordinates:
  * time         (time) datetime64[ns] 2020-10-02T08:48:25.419626 ... 2020-12...
  * y            (y) float64 6.821e+06 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x            (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref  int32 32734

* **To check all coordinates along a specific dimension**
<br>
<img src=https://live.staticflickr.com/65535/51115452191_ec160d4514_o.png, width="450">

In [5]:
ds.time
# OR
#ds.coords['time']

<xarray.DataArray 'time' (time: 19)>
array(['2020-10-02T08:48:25.419626000', '2020-10-07T08:48:23.287789000',
       '2020-10-12T08:48:25.481031000', '2020-10-17T08:48:23.388319000',
       '2020-10-22T08:48:25.013278000', '2020-10-27T08:48:22.864480000',
       '2020-11-01T08:48:24.722699000', '2020-11-06T08:48:21.688222000',
       '2020-11-11T08:48:23.663767000', '2020-11-16T08:48:20.487991000',
       '2020-11-21T08:48:21.902937000', '2020-11-26T08:48:19.711241000',
       '2020-12-01T08:48:19.324779000', '2020-12-06T08:48:18.060987000',
       '2020-12-11T08:48:16.179063000', '2020-12-16T08:48:15.553975000',
       '2020-12-21T08:48:18.337887000', '2020-12-26T08:48:17.064158000',
       '2020-12-31T08:48:19.761242000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-02T08:48:25.419626 ... 2020-12...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

* **To check attributes of the dataset**

In [6]:
ds.attrs

{'crs': 'EPSG:32734', 'grid_mapping': 'spatial_ref'}

## **Subset Dataset / DataArray**

* **To select all data of "blue" band**
<br>
<img src=https://live.staticflickr.com/65535/51115092614_366cb774a8_o.png, width="350">

In [7]:
ds.blue
# OR
#ds['blue']

<xarray.DataArray 'blue' (time: 19, y: 112, x: 197)>
array([[[  16,    8,    8, ..., 1782, 1740, 1256],
        [ 178,  192,  164, ..., 1696, 1764, 1676],
        [ 415,  375,  356, ..., 1190, 1344, 1640],
        ...,
        [1930, 2294, 2020, ...,  350,  338,  448],
        [1162, 1576, 2302, ...,  400,  350,  351],
        [1206, 1438, 1804, ...,  393,  350,  326]],

       [[ 306,  378,  367, ..., 1558, 1556, 1324],
        [ 262,  293,  284, ..., 1586, 1802, 1578],
        [ 440,  436,  401, ..., 1478, 1560, 1550],
        ...,
        [2024, 2196, 1752, ...,  414,  430,  454],
        [1412, 1742, 1968, ...,  415,  380,  385],
        [1238, 1192, 1730, ...,  453,  383,  345]],

       [[   9,    9,   28, ..., 1662, 1858, 1578],
        [ 121,  169,  107, ..., 1590, 1732, 1634],
        [ 293,  358,  360, ..., 1228, 1328, 1582],
        ...,
...
        ...,
        [6984, 6984, 6972, ..., 6816, 6852, 6852],
        [6996, 6996, 6984, ..., 6816, 6840, 6816],
        [6984, 7008, 6960, ..., 6772, 6772, 6820]],

       [[ 369,  337,  324, ..., 1636, 1514, 1300],
        [ 355,  346,  342, ..., 1800, 1776, 1386],
        [ 367,  363,  375, ..., 1734, 1822, 1860],
        ...,
        [2620, 2094, 1062, ...,  502,  490,  439],
        [1980, 2050, 2052, ...,  452,  440,  507],
        [1290, 1506, 2298, ...,  342,  387,  426]],

       [[ 373,  387,  383, ..., 1694, 1688, 1386],
        [ 338,  338,  367, ..., 1820, 1876, 1532],
        [ 412,  455,  429, ..., 1562, 1634, 1776],
        ...,
        [2374, 2380, 1718, ...,  612,  539,  545],
        [1580, 1890, 2344, ...,  503,  493,  568],
        [1440, 1236, 2030, ...,  386,  453,  455]]], dtype=int16)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-02T08:48:25.419626 ... 2020-12...
  * y            (y) float64 6.821e+06 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x            (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref  int32 32734
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

In [8]:
# Only print pixel values
ds.blue.values

array([[[  16,    8,    8, ..., 1782, 1740, 1256],
        [ 178,  192,  164, ..., 1696, 1764, 1676],
        [ 415,  375,  356, ..., 1190, 1344, 1640],
        ...,
        [1930, 2294, 2020, ...,  350,  338,  448],
        [1162, 1576, 2302, ...,  400,  350,  351],
        [1206, 1438, 1804, ...,  393,  350,  326]],

       [[ 306,  378,  367, ..., 1558, 1556, 1324],
        [ 262,  293,  284, ..., 1586, 1802, 1578],
        [ 440,  436,  401, ..., 1478, 1560, 1550],
        ...,
        [2024, 2196, 1752, ...,  414,  430,  454],
        [1412, 1742, 1968, ...,  415,  380,  385],
        [1238, 1192, 1730, ...,  453,  383,  345]],

       [[   9,    9,   28, ..., 1662, 1858, 1578],
        [ 121,  169,  107, ..., 1590, 1732, 1634],
        [ 293,  358,  360, ..., 1228, 1328, 1582],
        ...,
        [1704, 2156, 2256, ...,  178,  167,  170],
        [1202, 1304, 2138, ...,  298,  148,  158],
        [1312, 1142, 1708, ...,  270,  166,  142]],

       ...,

       [[7240, 7240, 720

* **To select blue band data at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51116131265_8464728bc1_o.png, width="350">

In [9]:
ds.blue[0]

<xarray.DataArray 'blue' (y: 112, x: 197)>
array([[  16,    8,    8, ..., 1782, 1740, 1256],
       [ 178,  192,  164, ..., 1696, 1764, 1676],
       [ 415,  375,  356, ..., 1190, 1344, 1640],
       ...,
       [1930, 2294, 2020, ...,  350,  338,  448],
       [1162, 1576, 2302, ...,  400,  350,  351],
       [1206, 1438, 1804, ...,  393,  350,  326]], dtype=int16)
Coordinates:
    time         datetime64[ns] 2020-10-02T08:48:25.419626
  * y            (y) float64 6.821e+06 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x            (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref  int32 32734
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select blue band data at the first time stamp while the latitude is the largest in the defined spatial extent**
<img src=https://live.staticflickr.com/65535/51115337046_aeb75d0d03_o.png, width="350">

In [10]:
ds.blue[0][0]

<xarray.DataArray 'blue' (x: 197)>
array([  16,    8,    8,    1,    1,   12,    6,   14,    4,    3,   15,
         19,    9,    7,    6,  127,  184,  106,  285,  661,  927, 1008,
        739,  512,  262,  233,  272,  297,  299,  213,  207,  277,  302,
        313,  287,  212,  167,  171,  203,  178,  165,  139,  177,  182,
        212,  317,  354,  299,  261,  256,  168,  110,  110,  144,  158,
        128,  175,  211,  455,  717,  437,  275,  287,  254,  270,  487,
        413,  252,  258,  429,  378,  463,  504,  855, 1088, 1076, 1044,
       1090, 1042, 1084, 1196, 1126,  919,  898, 1005, 1158, 1132,  938,
        926,  769,  732,  660,  258,  246,  242,  301,  401,  189,  319,
        292,  182,  446,  706,  431,  172,  251,  310,  333,  329,  307,
        296,  271,  274,  318,  522,  709,  642,  615,  611,  527,  557,
        782, 1001,  996,  819,  998, 1286, 1278, 1408,  971,  796,  669,
        563,  596,  834, 1146, 1190, 1208,  919,  868,  789,  866,  951,
        938,  859,  882,  898,  924,  972, 1042, 1266, 1182,  906,  851,
        858,  983, 1110, 1036,  963,  932,  923,  930,  904,  903,  899,
        765,  733,  706,  691,  694,  714,  801,  860,  772,  828,  810,
        827,  852,  871,  870,  882,  892,  948, 1001, 1444, 2058, 1696,
       1660, 1632, 1534, 1620, 1678, 1500, 1542, 1782, 1740, 1256],
      dtype=int16)
Coordinates:
    time         datetime64[ns] 2020-10-02T08:48:25.419626
    y            float64 6.821e+06
  * x            (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref  int32 32734
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select the upper-left corner pixel**
<br>
<img src=https://live.staticflickr.com/65535/51116131235_b0cca9589f_o.png, width="350">

In [11]:
ds.blue[0][0][0]

<xarray.DataArray 'blue' ()>
array(16, dtype=int16)
Coordinates:
    time         datetime64[ns] 2020-10-02T08:48:25.419626
    y            float64 6.821e+06
    x            float64 4.541e+05
    spatial_ref  int32 32734
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

### **subset dataset with `isel` vs. `sel`**
* Use `isel` when subsetting with **index**
* Use `sel` when subsetting with **labels**

* **To select data of all spectral bands at the first time stamp**
<br>
<img src=https://live.staticflickr.com/65535/51114879732_7d62db54f4_o.png, width="750">

In [12]:
ds.isel(time=[0])

<xarray.Dataset>
Dimensions:          (time: 1, x: 197, y: 112)
Coordinates:
  * time             (time) datetime64[ns] 2020-10-02T08:48:25.419626
  * y                (y) float64 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x                (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref      int32 32734
Data variables: (12/13)
    coastal_aerosol  (time, y, x) int16 222 222 222 222 192 ... 305 305 305 287
    blue             (time, y, x) int16 16 8 8 1 1 12 ... 748 594 393 350 326
    green            (time, y, x) int16 203 219 213 254 243 ... 1096 951 923 783
    red              (time, y, x) int16 148 160 120 118 124 ... 1009 775 618 672
    red_edge1        (time, y, x) int16 450 450 509 509 ... 2046 1593 1593 1355
    red_edge2        (time, y, x) int16 738 738 442 442 ... 3581 3768 3768 2466
    ...               ...
    nir              (time, y, x) int16 170 159 1 1 40 ... 4048 4212 4276 3584
    narrow_nir       (time, y, x) int16 998 998 874 874 ... 3948 4187 4187 2965
    water_vapour     (time, y, x) int16 1862 1862 1862 1862 ... 4201 4201 3413
    swir1            (time, y, x) int16 711 711 742 742 ... 2866 2504 2504 2187
    swir2            (time, y, x) int16 471 471 431 431 ... 2019 1685 1685 1597
    scl              (time, y, x) uint8 2 2 2 2 2 2 2 2 6 ... 4 4 4 4 4 4 4 4 4
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **To select data of all spectral bands of year 2020** 
<br>
<img src=https://live.staticflickr.com/65535/51116281070_75f1b46a9c_o.png, width="750">

In [13]:
ds.sel(time='2020-12')
#print(ds.sel(time='2019'))

<xarray.Dataset>
Dimensions:          (time: 7, x: 197, y: 112)
Coordinates:
  * time             (time) datetime64[ns] 2020-12-01T08:48:19.324779 ... 202...
  * y                (y) float64 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x                (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref      int32 32734
Data variables: (12/13)
    coastal_aerosol  (time, y, x) int16 275 275 275 275 248 ... 287 287 287 289
    blue             (time, y, x) int16 240 193 204 199 183 ... 510 386 453 455
    green            (time, y, x) int16 508 517 501 451 468 ... 1015 858 868 869
    red              (time, y, x) int16 347 379 362 322 326 ... 1042 693 861 913
    red_edge1        (time, y, x) int16 521 521 459 459 ... 1753 1500 1500 1278
    red_edge2        (time, y, x) int16 607 607 957 957 ... 2971 3150 3150 3121
    ...               ...
    nir              (time, y, x) int16 220 690 390 50 ... 3708 3934 3780 3492
    narrow_nir       (time, y, x) int16 766 766 1025 1025 ... 3927 3927 3958
    water_vapour     (time, y, x) int16 2577 2577 2577 2577 ... 4119 4119 4294
    swir1            (time, y, x) int16 679 679 638 638 ... 2919 2796 2796 2342
    swir2            (time, y, x) int16 379 379 349 349 ... 1888 1714 1714 1368
    scl              (time, y, x) uint8 2 2 2 2 2 2 2 2 6 ... 4 4 4 4 4 4 4 4 4
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

***Tip: More about indexing and sebsetting Dataset or DataArray is presented in the [Notebook_05](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/05_xarrayII.ipynb).***

## **Reshape Dataset**

* **Convert the Dataset (subset to 2019) to a *4-dimension* DataArray**

In [14]:
da = ds.sel(time='2020-12').to_array().rename({"variable":"band"})
da

<xarray.DataArray (band: 13, time: 7, y: 112, x: 197)>
array([[[[ 275,  275,  275, ..., 1126, 1126, 1083],
         [ 275,  275,  275, ..., 1126, 1126, 1083],
         [ 240,  240,  240, ..., 1234, 1234, 1196],
         ...,
         [ 881,  881,  881, ...,  158,  158,  184],
         [1158, 1158, 1158, ...,  163,  163,  200],
         [1158, 1158, 1158, ...,  163,  163,  200]],

        [[ 283,  283,  283, ..., 1007, 1007,  978],
         [ 283,  283,  283, ..., 1007, 1007,  978],
         [ 259,  259,  259, ..., 1112, 1112, 1074],
         ...,
         [ 798,  798,  798, ...,  254,  254,  300],
         [1062, 1062, 1062, ...,  185,  185,  226],
         [1062, 1062, 1062, ...,  185,  185,  226]],

        [[ 232,  232,  232, ..., 1075, 1075, 1042],
         [ 232,  232,  232, ..., 1075, 1075, 1042],
         [ 247,  247,  247, ..., 1108, 1108, 1081],
         ...,
...
         ...,
         [   9,    9,    9, ...,    9,    9,    9],
         [   9,    9,    9, ...,    9,    9,    9],
         [   9,    9,    9, ...,    9,    9,    9]],

        [[   2,    2,    2, ...,    5,    5,    5],
         [   2,    2,    2, ...,    5,    5,    5],
         [   4,    4,    7, ...,    5,    5,    5],
         ...,
         [   5,    5,    4, ...,    4,    4,    4],
         [   5,    5,    5, ...,    4,    4,    4],
         [   5,    5,    5, ...,    4,    4,    4]],

        [[   2,    2,    2, ...,    5,    5,    5],
         [   2,    2,    2, ...,    5,    5,    5],
         [   4,    4,    4, ...,    5,    5,    5],
         ...,
         [   5,    5,    4, ...,    4,    4,    4],
         [   5,    5,    5, ...,    4,    4,    4],
         [   5,    5,    5, ...,    4,    4,    4]]]], dtype=int16)
Coordinates:
  * time         (time) datetime64[ns] 2020-12-01T08:48:19.324779 ... 2020-12...
  * y            (y) float64 6.821e+06 6.821e+06 6.821e+06 ... 6.82e+06 6.82e+06
  * x            (x) float64 4.541e+05 4.541e+05 ... 4.561e+05 4.561e+05
    spatial_ref  int32 32734
  * band         (band) <U15 'coastal_aerosol' 'blue' 'green' ... 'swir2' 'scl'
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

* **Convert the *4-dimension* DataArray back to a Dataset by setting the "time" as DataVariable (reshaped)**

![ds_reshaped](https://live.staticflickr.com/65535/51151694092_ca550152d6_o.png)

In [15]:
ds_reshp = da.to_dataset(dim="time")
print(ds_reshp)

<xarray.Dataset>
Dimensions:                     (band: 13, x: 197, y: 112)
Coordinates:
  * y                           (y) float64 6.821e+06 6.821e+06 ... 6.82e+06
  * x                           (x) float64 4.541e+05 4.541e+05 ... 4.561e+05
    spatial_ref                 int32 32734
  * band                        (band) <U15 'coastal_aerosol' 'blue' ... 'scl'
Data variables:
    2020-12-01 08:48:19.324779  (band, y, x) int16 275 275 275 275 ... 4 4 4 4
    2020-12-06 08:48:18.060987  (band, y, x) int16 283 283 283 283 ... 4 4 4 4
    2020-12-11 08:48:16.179063  (band, y, x) int16 232 232 232 232 ... 4 4 4 4
    2020-12-16 08:48:15.553975  (band, y, x) int16 177 177 177 177 ... 5 5 5 4
    2020-12-21 08:48:18.337887  (band, y, x) int16 6966 6966 6966 6966 ... 9 9 9
    2020-12-26 08:48:17.064158  (band, y, x) int16 231 231 231 231 ... 4 4 4 4
    2020-12-31 08:48:19.761242  (band, y, x) int16 277 277 277 277 ... 4 4 4 4
Attributes:
    crs:           EPSG:32734
    grid_mapping:  s

## Recommended next steps

If you now understand the **data structure** of `xarray.dataset` and **basic indexing** methods illustrated in this notebook, you are ready to move on to the next notebook where you will learn more about **advanced indexing** and calculating some **basic statistical parameters** of the n-dimensional arrays!:D

In case you are gaining interest in exploring the world of **xarrays**, you may lay yourself into the [Xarray user guide](http://xarray.pydata.org/en/stable/index.html).

<br>
To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/01_jupyter_introduction.ipynb)
2. [eo2cube](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/02_eo2cube_introduction.ipynb)
3. [Loading Data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/03_data_lookup_and_loading.ipynb)
4. ***Xarray I: Data Structure (this notebook)***
5. [Xarray II: Index and Statistics](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/05_xarrayII.ipynb)
6. [Plotting data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/06_plotting_basics.ipynb)
7. [Spatial analysis](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/07_basic_analysis.ipynb)
8. [Parallel processing with Dask](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/08_parallel_processing_with_dask.ipynb)

The additional notebooks are designed for users to build up both basic and advanced skills which are not covered by the beginner's guide. Self-motivated users can go through them according to their own needs. They act as complements for the guide:
<br>

1. [Python's file management tools](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/I_file_management.ipynb)
2. [Image Processing basics using NumPy and Matplotlib](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/II_numpy_image_processing.ipynb)
3. [Vector Processing](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/III_process_vector_data.ipynb)
4. [Advanced Plotting](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/IV_advanced_plotting.ipynb)

***
## Additional information

This notebook is for the usage of Jupyter Notebook of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/).

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** April 2021